# Investing.com Crawling
---

### 1. 종목 마스터 정보
Target: https://www.investing.com/indices/indices-futures 의 Specification 탭

In [1]:
# -*- coding: utf-8 -*-
import sys
import requests
import re

indi_fut_url = "https://www.investing.com/indices/indices-futures"

headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Cache-Control": "max-age=0",
    "Connection": "keep-alive",
    "Host": "www.investing.com",
    "Referer": "https://www.investing.com/",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36",
}

resp = requests.get(url=indi_fut_url, headers=headers)

pattern = re.compile("<.+?type=\"Specifications\".+?execPricePerfTechButton\((.+?)\).+?>")
find_strs = pattern.findall(resp.text)
js_args = find_strs[0].replace(" ", "").replace("\'", "").split(",")

pattern = re.compile("window.siteData.smlID.*?=(.+?);")
find_strs = pattern.findall(resp.text)
sml_id = find_strs[0].strip()

print("js_args:", js_args)
print("sml_id:", sml_id)

js_args: ['this', 'indices', 'cross_rates_container', 'Futures', '0', '56ed2c921d6ef28c273007be75597f06']
sml_id: 70


In [2]:
spc_fut_url = f"https://www.investing.com/{js_args[1]}/Service/SpecificationsFutures"

headers = {
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Connection": "keep-alive",
    "Host": "www.investing.com",
    "Referer": "https://www.investing.com/indices/indices-futures",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-origin",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
}

params = {
    "pairid": js_args[4],
    "sid": js_args[5],
    "filterParams": "",
    "smlID": sml_id
}

resp = requests.get(url=spc_fut_url, headers=headers, params=params)

print("spc_fut_url:", spc_fut_url)
print("params:", params)

spc_fut_url: https://www.investing.com/indices/Service/SpecificationsFutures
params: {'pairid': '0', 'sid': '56ed2c921d6ef28c273007be75597f06', 'filterParams': '', 'smlID': '70'}


In [3]:
from bs4 import BeautifulSoup

masters = []

bs = BeautifulSoup(resp.text, "html.parser")

spc_tbl = bs.find("table", id="specifications")
tbody = spc_tbl.find("tbody")
trs = tbody.find_all("tr")

for tr in trs:
    tds = tr.find_all("td")

    data = {
        "pid": tr.get("pair-id"),
        "trd_hour": tr.get("data-trade-ours"),
        "last_trd_day": tr.get("data-last-trade"),
        "tick_size": tr.get("data-tick-size"),
        "tick_value": tr.get("data-tick-value"),
        "stmt_type": tr.get("data-settl-type"),
        "country": tds[0].find("span").get("title"),
        "item_nm": tds[1].find("a").get("title"),
        "symbol": tds[2].get_text(),
        "exchange": tds[3].get_text(),
        "contract_size": tds[4].get_text(),
        "month": tds[5].get_text(),
        "point_value": tds[6].get_text(),
    }

    masters.append(data)

print(masters)

[{'pid': '8873', 'trd_hour': '17:00 – 16:15 CT (Sunday – Friday)', 'last_trd_day': 'The third Friday of the contract month', 'tick_size': '1', 'tick_value': '5', 'stmt_type': 'Cash', 'country': 'United States', 'item_nm': 'Dow Jones Futures (CFD)', 'symbol': 'YM', 'exchange': '', 'contract_size': '$5 x Dow Jones', 'month': 'HMUZ', 'point_value': '1 = $5'}, {'pid': '8839', 'trd_hour': '17:00 – 16:15 CT (Sunday – Friday)', 'last_trd_day': 'The third Friday of the contract month', 'tick_size': '0.25', 'tick_value': '12.5', 'stmt_type': 'Cash', 'country': 'United States', 'item_nm': 'S&P 500 Futures (CFD)', 'symbol': 'ES', 'exchange': '', 'contract_size': '$50 x Index Price', 'month': 'HMUZ', 'point_value': '1 = $50'}, {'pid': '8874', 'trd_hour': '17:00 - 16:15 CT (Monday - Friday)', 'last_trd_day': 'The third Friday of the contract month', 'tick_size': '0.25', 'tick_value': '5', 'stmt_type': 'Cash', 'country': 'United States', 'item_nm': 'NASDAQ Futures (CFD)', 'symbol': 'NQ', 'exchange':

### 2. 선물 만기정보
Target: https://www.investing.com/futures-expiration-calendar/

In [4]:
import requests
import json

base_url = "https://www.investing.com/futures-expiration-calendar/Service/getCalendarFilteredData"

headers = {
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Connection": "keep-alive",
    "Content-Length": "49",
    "Content-Type": "application/x-www-form-urlencoded",
    "Host": "www.investing.com",
    "Origin": "https://www.investing.com",
    "Referer": "https://www.investing.com/futures-expiration-calendar/",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-origin",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest",
}

param_query = {
    "currentTab": "thisMonth",
    "submitFilters": "1",
    "limit_from": "0",
}

resp = requests.post(url=base_url, data=param_query, headers=headers)

print("status_code:", resp.status_code)

status_code: 200


In [5]:
from bs4 import BeautifulSoup

def str_to_int(val: str, default_val: int = None) -> int:
    return int(val) if val.isdigit() else default_val

exp_infos = []

rcv_dict = json.loads(resp.text)
bs = BeautifulSoup(rcv_dict.get("data"), "html.parser")

trs = bs.find_all("tr")

for tr in trs:
    tds = tr.find_all("td")

    code = tr.get("id")

    if code is not None:
        code = code.replace("inst_", "")
        exp_ts = str_to_int(tds[2].get("data-value"))
        settlement_ts = str_to_int(tds[3].get("data-value"))
        last_rollover_ts = str_to_int(tds[4].get("data-value"))

        # 선물 만기 정보
        exp_infos.append({
            "code": code,
            "exp_ts": exp_ts,
            "settlement_ts": settlement_ts,
            "last_rollover_ts": last_rollover_ts
        })

print(exp_infos)

[{'code': '13917', 'exp_ts': 1608422400, 'settlement_ts': None, 'last_rollover_ts': 1599350400}, {'code': '104417', 'exp_ts': 1608422400, 'settlement_ts': 1545264000, 'last_rollover_ts': 1599955200}, {'code': '961741', 'exp_ts': 1579564800, 'settlement_ts': 1547510400, 'last_rollover_ts': 1599350400}, {'code': '941609', 'exp_ts': 1608422400, 'settlement_ts': 1599696000, 'last_rollover_ts': 1599436800}, {'code': '1073168', 'exp_ts': 1600560000, 'settlement_ts': 1600300800, 'last_rollover_ts': None}, {'code': '1152915', 'exp_ts': 1600560000, 'settlement_ts': 1600387200, 'last_rollover_ts': None}, {'code': '1152920', 'exp_ts': 1600560000, 'settlement_ts': 1600387200, 'last_rollover_ts': None}, {'code': '1152925', 'exp_ts': 1600560000, 'settlement_ts': 1600387200, 'last_rollover_ts': None}, {'code': '1152913', 'exp_ts': 1600560000, 'settlement_ts': 1600387200, 'last_rollover_ts': None}, {'code': '1152924', 'exp_ts': 1600560000, 'settlement_ts': 1600387200, 'last_rollover_ts': None}, {'code

### 3. 선물 실시간 시세
Target: https://www.investing.com/indices/indices-futures

In [6]:
import requests
import re
import random

indi_fut_url = "https://www.investing.com/indices/indices-futures"

headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Cache-Control": "max-age=0",
    "Connection": "keep-alive",
    "Host": "www.investing.com",
    "Referer": "https://www.investing.com/",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36",
}

resp = requests.get(url=indi_fut_url, headers=headers)

pattern = re.compile("window.stream.+?\"(.+?)\"")
find_strs = pattern.findall(resp.text)
base_ws_url = find_strs[0]

pattern = re.compile("TimeZoneID.+?\"(.+?)\"")
find_strs = pattern.findall(resp.text)
timezone_id = find_strs[0]

rnd_num = "%03d" % random.randint(0, 999)

rnd_str = ""
for i in range(8):
    rnd_str = rnd_str + random.choice("abcdefghijklmnopqrstuvwxyz0123456789_")

ws_url = base_ws_url + "/echo/" + rnd_num + "/" + rnd_str + "/websocket"
ws_url = "wss" + ws_url[5:] if "https" == ws_url[:5] else "ws" + ws_url[4:]

print("Websocket URL:", ws_url)

Websocket URL: wss://stream237.forexpros.com:443/echo/7331/dgqei70t/websocket


In [7]:
import threading
import json
import time
import ssl
import websocket

def heartbeat(ws):
    time.sleep(5)

    heartbeat_obj = {
        "_event": "heartbeat",
        "data": "h"
    }
    heartbeat_msg = json.dumps(heartbeat_obj)
    heartbeat_msg = json.dumps([heartbeat_msg])

    while True:
        try:
            time.sleep(3)
            ws.send(heartbeat_msg)
        except Exception as e:
            print(e)
            break


def on_message(ws, msg) -> None:
    if "o" == msg:
        # subscribe
        items = ["pid-8873:", "pid-8839:", "pid-8874:", "pid-8864:", "pid-8884:", "pid-8826:", "pid-8853:", "pid-8838:", "pid-8867:", "pid-8858:", "pid-8837:", "pid-8828:", "pid-8825:", "pid-8863:", "pid-8878:", "pid-8882:", "pid-104396:", "pid-961616:", "pid-941612:", "pid-8859:", "pid-941609:", "pid-8984:", "pid-8982:", "pid-941616:", "pid-44486:", "pid-8824:", "pid-8897:", "pid-8985:", "pid-104423:", "pid-8987:", "pid-104392:", "pid-8840:", "pid-15288:", "pid-1090480:", "pid-40717:", "pid-40744:", "pid-40703:", "pid-40745:", "pid-40742:", "pid-40715:", "pid-40724:", "pid-40792:", "pid-40793:", "pid-40725:", "pid-40794:", "pid-40726:", "pid-40796:", "pid-40797:", "pid-40798:", "pid-40799:", "pid-40800:", "pid-40801:", "pid-40802:", "pid-40803:", "pid-40804:", "pid-40805:", "pid-40806:", "pid-40807:", "pid-40809:", "pid-40810:", "pid-40811:", "pid-40812:", "pid-40813:", "pid-40814:", "pid-40815:", "pid-40816:", "pid-40714:", "pid-40765:", "pid-40766:", "pid-40767:", "pid-40768:", "pid-40769:", "pid-40770:", "pid-40771:", "pid-40772:", "pid-40773:", "pid-40774:", "pid-40775:", "pid-40776:", "pid-40777:", "pid-40778:", "pid-40728:", "pid-40779:", "pid-40780:", "pid-40781:", "pid-40782:", "pid-40783:", "pid-40743:", "pid-40727:", "pid-40784:", "pid-40785:", "pid-40786:", "pid-40787:", "pid-40788:", "pid-40789:", "pid-40790:", "pid-40791:", "pid-40808:", "pid-40764:", "pid-40756:", "pid-40718:", "pid-40758:", "pid-40723:", "pid-40716:", "pid-40759:", "pid-40729:", "pid-40731:", "pid-40763:", "pid-40730:", "pid-40719:", "pid-40817:", "pid-169:", "pid-166:", "pid-14958:", "pid-44336:", "pid-8827:", "pid-1:", "pid-3:", "pid-2:", "pid-5:", "pid-9:", "pid-7:", "pid-10:", "pid-998471:"]
        subs_msg = {"_event": "bulk-subscribe", "tzID": timezone_id, "message": "%%".join(items)}
        subs_msg = json.dumps(subs_msg)
        subs_msg = json.dumps([subs_msg])
        ws.send(subs_msg)

        # UID
        uids = {"_event": "UID", "UID": 0}
        uids_msg = json.dumps(uids)
        uids_msg = json.dumps([uids_msg])
        ws.send(uids_msg)

        # heartbeat
        heartbeat_thd = threading.Thread(target=heartbeat, args=(ws,))
        heartbeat_thd.start()


def on_data(ws, m, t, c) -> None:
    if type(m) == str and 0 < len(m) and "a" == m[0]:
        rcv_msg = json.loads(m[1:])
        rcv_msg = json.loads(rcv_msg[0])

        if "message" in rcv_msg:
            rcv_msg = rcv_msg["message"]
            pattern = re.compile("::(.+)")
            rcv_msg = pattern.findall(rcv_msg)[0]
            rcv_msg = json.loads(rcv_msg)

        print(rcv_msg)


def on_error(ws, error) -> None:
    print(error)


def on_close(ws) -> None:
    print("### close ###")


def on_open(ws) -> None:
    print("### open ###")


websocket.enableTrace(True)
ws = websocket.WebSocketApp(ws_url,
                            on_message=on_message,
                            on_data=on_data,
                            on_error=on_error,
                            on_close=on_close,
                            on_open=on_open)
ws.run_forever(sslopt={"cert_reqs": ssl.CERT_NONE})

--- request header ---
GET /echo/7331/dgqei70t/websocket HTTP/1.1
Upgrade: websocket
Connection: Upgrade
Host: stream237.forexpros.com
Origin: http://stream237.forexpros.com
Sec-WebSocket-Key: EDe0ZLhIWWC/wajz61xWAQ==
Sec-WebSocket-Version: 13


-----------------------
--- response header ---
HTTP/1.1 101 Switching Protocols
Upgrade: websocket
Connection: Upgrade
Sec-WebSocket-Accept: p/oknnlijf487oaHFb4apuFrqgA=
-----------------------
send: b'\x81\xfe\x05\xe8&\xc4>\x01}\xe6E]\x04\x9b[wC\xaaJ]\x04\xfe\x1e]\x04\xa6KmM\xe9MtD\xb7]sO\xa6[]\x04\xe8\x1e]\x04\xb0DHb\x98\x1c;\x06\x98\x1c9z\xe6\x12!z\xe6SdU\xb7_fC\x98\x1c;\x06\x98\x1cqO\xa0\x139\x1e\xf3\r;\x03\xe1NhB\xe9\x069\x15\xfd\x04$\x03\xb4We\x0b\xfc\x066\x12\xfe\x1b$V\xadZ,\x1e\xfc\x085\x1c\xe1\x1bqO\xa0\x139\x1e\xfc\n;\x03\xe1NhB\xe9\x069\x14\xf2\x04$\x03\xb4We\x0b\xfc\x064\x15\xfe\x1b$V\xadZ,\x1e\xfc\r9\x1c\xe1\x1bqO\xa0\x139\x1e\xf2\t;\x03\xe1NhB\xe9\x069\x13\xfc\x04$\x03\xb4We\x0b\xfc\x062\x11\xfe\x1b$V\xadZ,\x1e\xfc\x0c9\x1c\xe1\x

### open ###
{'pid': '8859', 'last_dir': 'greenBg', 'last_numeric': 23118, 'last': '23,118.0', 'bid': '23,106.0', 'ask': '23,130.0', 'high': '23,258.0', 'low': '23,102.5', 'pc': '+18.0', 'pcp': '+0.08%', 'pc_col': 'greenFont', 'time': '15:39:53', 'timestamp': 1600443593}
{'pid': '2', 'last_dir': 'greenBg', 'last_numeric': 1.2944, 'last': '1.2944', 'bid': '1.2944', 'ask': '1.2945', 'high': '1.3000', 'low': '1.2934', 'last_close': '1.2972', 'pc': '-0.0028', 'pcp': '-0.21%', 'pc_col': 'redFont', 'turnover': '127.71K', 'turnover_numeric': '127705', 'time': '15:39:53', 'timestamp': 1600443592}
{'pid': '5', 'last_dir': 'redBg', 'last_numeric': 0.7309, 'last': '0.7309', 'bid': '0.7308', 'ask': '0.7311', 'high': '0.7334', 'low': '0.7283', 'last_close': '0.7312', 'pc': '-0.0002', 'pcp': '-0.03%', 'pc_col': 'redFont', 'turnover': '63.08K', 'turnover_numeric': '63079', 'time': '15:39:53', 'timestamp': 1600443591}
{'pid': '7', 'last_dir': 'greenBg', 'last_numeric': 1.318, 'last': '1.3180', 'bid': 

send: b'\x88\x825\x8c\xec[6d'


{'pid': '8859', 'last_dir': 'greenBg', 'last_numeric': 23123, 'last': '23,123.0', 'bid': '23,111.0', 'ask': '23,135.0', 'high': '23,258.0', 'low': '23,102.5', 'pc': '+23.0', 'pcp': '+0.10%', 'pc_col': 'greenFont', 'time': '15:39:58', 'timestamp': 1600443598}
{'pid': '15288', 'last_dir': '', 'last_numeric': 3125.5, 'last': '3,125.50', 'bid': '3,104.00', 'ask': '3,138.00', 'high': '3,143.50', 'low': '3,077.50', 'pc': '+48.00', 'pcp': '+1.56%', 'pc_col': 'greenFont', 'time': '15:39:58', 'timestamp': 1600443594}

### close ###


False

Connection is already closed.
